In [1]:
from random import randint
from pandas import DataFrame,options,ExcelWriter,read_csv
from IPython.display import clear_output

In [2]:
options.display.float_format = '{:,}'.format

In [3]:
def set_state_polls(trump_error:int, biden_error:int) -> dict:
    state_polls = { # State:[Trump,Biden],
        'Alabama':[57,38],
        'Alaska':[53,41],
        'Arizona':[47,43],
        'Arkansas':[57,24],
        'California':[31,55],
        'Colorado':[39,49],
        'Connecticut':[40,49],
        'Delaware':[36,46],
        'District of Columbia':[10,89],
        'Florida':[49,43],
        'Georgia':[45,38],
        'Hawaii':[38,57],
        'Idaho':[54,25],
        'Illinois':[34,43],
        'Indiana':[55,34],
        'Iowa':[50,32],
        'Kansas':[47,31],
        'Kentucky':[55,26],
        'Louisiana':[52,38],
        'Maine':[41,40],
        'Maryland':[30,56],
        'Massachusetts':[27,52],
        'Michigan':[44,44],
        'Minnesota':[43,45],
        'Mississippi':[50,35],
        'Missouri':[53,40],
        'Montana':[54,32],
        'Nebraska':[47,31],
        'Nevada':[47,42],
        'New Hampshire':[42,44],
        'New Jersey':[41,40],
        'New Mexico':[41,48],
        'New York':[40,48],
        'North Carolina':[44,38],
        'North Dakota':[62,28],
        'Ohio':[51,44],
        'Oklahoma':[55,27],
        'Oregon':[29,40],
        'Pennsylvania':[47,44],
        'Rhode Island':[33,40],
        'South Carolina':[52,36],
        'South Dakota':[50,31],
        'Tennessee':[47,29],
        'Texas':[47,41],
        'Utah':[57,25],
        'Vermont':[29,47],
        'Virginia':[44,44],
        'Washington':[37,49],
        'West Virginia':[59,23],
        'Wisconsin':[46,47],
        'Wyoming':[68,15]
    }
    updated_polls = {state: [val1 + trump_error, val2 + biden_error] for state, (val1, val2) in state_polls.items()}
    state_polls = updated_polls.copy()
    return state_polls

In [4]:
def reset_variables(states:list, candidates:dict):
    trump_states_won = []
    biden_states_won = []
    electoral_votes_results = {candidate:0 for candidate in candidates}
    popular_vote_results = {candidate:0 for candidate in candidates}
    state_vote_counts = {state:[0,0,0] for state in states} # state:[Total,Trump,Biden]
    return trump_states_won,biden_states_won,electoral_votes_results,popular_vote_results,state_vote_counts

In [5]:
def get_individual_state_votes(state:str, registered_voters:dict, state_polls:dict, states:list, electoral_votes:dict,
                               popular_vote_results:dict, state_vote_counts:dict, electoral_votes_results:dict,
                               trump_states_won:list, biden_states_won:list, trump_error:int, biden_error:int,iteration:int):
    clear_output(wait=False)
    print(f"Current State: {state}\nStates Completed: {states.index(state)}\nStates Left: {len(states)-states.index(state)}\nTrump Error: {trump_error}\nBiden Error: {biden_error}\nRound: {iteration}")
    trump_state_votes = 0 
    biden_state_votes = 0
    for _ in range(registered_voters[state]):
        voter_choice = randint(1,sum(state_polls[state])+3)
        voter_switches = randint(1,1000)
        actively_voting = randint(1,100)
        if(
            (
                (voter_choice<=state_polls[state][0])and # Voter's choice is for Trump
                (actively_voting<=80)and # Voter is going to cast a ballot 80% of the time
                (voter_switches<=990) # Voter does not switch their vote at the last minute
            )
            or
            (
                (voter_choice>state_polls[state][0])and # Voter's choice is not for Trump
                (voter_choice<=sum(state_polls[state]))and # Voter's choice is for Biden
                (actively_voting<=80)and # Voter is going to cast a ballot 80% of the time
                (voter_switches>990 and voter_switches<997) # Voter decided at the last second to switch their vote for Trump
            )
          ):
            trump_state_votes += 1
            popular_vote_results['Donald Trump'] += 1
            popular_vote_results['Total'] += 1
            state_vote_counts[state][0] += 1
            state_vote_counts[state][1] += 1
        elif(
                (
                    (voter_choice<=state_polls[state][0])and # Voter's choice is for Trump
                    (actively_voting<=80)and # Voter is going to cast a ballot 80% of the time
                    (voter_switches>990 and voter_switches<997) # Voter decided at the last second to switch their vote for Biden
                )
                or
                (
                    (voter_choice>state_polls[state][0])and # Voter's choice is not for Trump
                    (voter_choice<=sum(state_polls[state]))and # Voter's choice is for Biden
                    (actively_voting<=80)and # Voter is going to cast a ballot 80% of the time
                    (voter_switches<=990) # Voter does not switch their vote at the last minute
                )
            ):
            biden_state_votes += 1
            popular_vote_results['Joe Biden'] += 1
            popular_vote_results['Total'] += 1
            state_vote_counts[state][0] += 1
            state_vote_counts[state][2] += 1
        elif(
                (
                    (voter_choice<=state_polls[state][0])and # Voter's choice is for Trump
                    (actively_voting<=80)and # Voter is going to cast a ballot 80% of the time
                    (voter_switches>=997) # Voter decided at the last second to vote for a third party
                )
                or
                (
                    (voter_choice>state_polls[state][0])and # Voter's choice is not for Trump
                    (voter_choice<=sum(state_polls[state]))and # Voter's choice is for Biden
                    (actively_voting<=80)and # Voter is going to cast a ballot 80% of the time
                    (voter_switches>=997) # Voter decided at the last second to vote for a third party
                )
                or
                (
                    (voter_choice>sum(state_polls[state]))and # Voter's choice is neither Trump or Biden
                    (actively_voting<=80)and # Voter is going to cast a ballot 80% of the time
                    (voter_switches<=990) # Voter does not switch their vote at the last minute
                )
            ):
            popular_vote_results['Total'] += 1
            state_vote_counts[state][0] += 1
    if(trump_state_votes>biden_state_votes):
        electoral_votes_results['Donald Trump'] += electoral_votes[state]
        trump_states_won.append(state)
    elif(trump_state_votes<biden_state_votes):
        electoral_votes_results['Joe Biden'] += electoral_votes[state]
        biden_states_won.append(state)
    else:
        get_individual_state_votes(state, registered_voters, state_polls, states, popular_vote_results, 
                               state_vote_counts, electoral_votes_results,trump_states_won,electoral_votes,
                               biden_states_won)
    return trump_states_won,biden_states_won,electoral_votes_results,popular_vote_results,state_vote_counts

In [6]:
def get_state_results_df(states:list, electoral_votes:dict, trump_states_won:list, state_vote_counts:dict) -> DataFrame: 
    detailed_results:list = []
    for state in states:
        row:list = []
        row.append(state)
        row.append(electoral_votes[state])
        if state in trump_states_won:
            row.append('Trump')
        else:
            row.append('Biden')
        row.append(state_vote_counts[state][0])
        row.append(state_vote_counts[state][1])
        row.append(state_vote_counts[state][2])
        detailed_results.append(row)
    state_results_df: DataFrame = DataFrame(detailed_results,columns=['State','Electoral_Votes','Winner','Total_Vote_Count','Trump_Vote_Count','Biden_Vote_Count'])
    state_results_df['Trump_Vote_Percent'] = round((state_results_df['Trump_Vote_Count']/(state_results_df['Total_Vote_Count']))*100,3)
    state_results_df['Biden_Vote_Percent'] = round((state_results_df['Biden_Vote_Count']/(state_results_df['Total_Vote_Count']))*100,3)
    return state_results_df

In [7]:
def get_national_results_df(electoral_votes_results:dict, popular_vote_results:dict, election_scores:DataFrame,
                            trump_error:int, biden_error:int, iteration:int) -> list:
    national_results_df: DataFrame = DataFrame([[electoral_votes_results['Donald Trump'],
                                    electoral_votes_results['Joe Biden'],
                                    popular_vote_results['Donald Trump'],
                                    popular_vote_results['Joe Biden'],
                                    round((popular_vote_results['Donald Trump']/(popular_vote_results['Total']))*100,3),
                                    round((popular_vote_results['Joe Biden']/(popular_vote_results['Total']))*100,3)]],
                                    columns=['Trump_Electoral_Votes','Biden_Electoral_Votes','Trump_Total_Votes','Biden_Total_Votes','Trump_Vote_Percent','Biden_Vote_Percent'])
    if(electoral_votes_results['Donald Trump']>electoral_votes_results['Joe Biden']):
        election_scores.loc[len(election_scores)] = ['Donald Trump',electoral_votes_results['Donald Trump'],electoral_votes_results['Joe Biden'],trump_error,biden_error,iteration]
        election_scores.to_csv('Summarized_Election_Scores.csv')
    elif(electoral_votes_results['Donald Trump']<electoral_votes_results['Joe Biden']):
        election_scores.loc[len(election_scores)] = ['Joe Biden',electoral_votes_results['Donald Trump'],electoral_votes_results['Joe Biden'],trump_error,biden_error,iteration]
        election_scores.to_csv('Summarized_Election_Scores.csv')
    else:
        election_scores.loc[len(election_scores)] = ['TIE',electoral_votes_results['Donald Trump'],electoral_votes_results['Joe Biden'],trump_error,biden_error,iteration]
        election_scores.to_csv('Summarized_Election_Scores.csv')
    return [national_results_df,election_scores]

In [8]:
states:list = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", 
    "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", 
    "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", 
    "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", 
    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", 
    "New Hampshire", "New Jersey", "New Mexico", "New York", 
    "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", 
    "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", 
    "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", 
    "West Virginia", "Wisconsin", "Wyoming", "District of Columbia"
]

In [9]:
electoral_votes:dict = {
    'Alabama': 9, 'Alaska': 3, 'Arizona': 11, 'Arkansas': 6, 'California': 54,
    'Colorado': 10, 'Connecticut': 7, 'Delaware': 3, 'Florida': 30, 'Georgia': 16,
    'Hawaii': 4, 'Idaho': 4, 'Illinois': 19, 'Indiana': 11, 'Iowa': 6,
    'Kansas': 6, 'Kentucky': 8, 'Louisiana': 8, 'Maine': 4, 'Maryland': 10,
    'Massachusetts': 11, 'Michigan': 15, 'Minnesota': 10, 'Mississippi': 6, 'Missouri': 10,
    'Montana': 4, 'Nebraska': 5, 'Nevada': 6, 'New Hampshire': 4, 'New Jersey': 14,
    'New Mexico': 5, 'New York': 28, 'North Carolina': 16, 'North Dakota': 3, 'Ohio': 17,
    'Oklahoma': 7, 'Oregon': 8, 'Pennsylvania': 19, 'Rhode Island': 4, 'South Carolina': 9,
    'South Dakota': 3, 'Tennessee': 11, 'Texas': 40, 'Utah': 6, 'Vermont': 3,
    'Virginia': 13, 'Washington': 12, 'West Virginia': 4, 'Wisconsin': 10, 'Wyoming': 3,
    'District of Columbia': 3
}

In [10]:
registered_voters:dict = {
    'Alabama': 2_499_000, 'Alaska': 373_000, 'Arizona': 3_560_000, 'Arkansas': 1_360_000, 'California': 17_032_000,
    'Colorado': 3_162_000, 'Connecticut': 1_778_000, 'Delaware': 578_000, 'Florida': 9_770_000, 'Georgia': 5_275_000,
    'Hawaii': 651_000, 'Idaho': 917_000, 'Illinois': 6_110_000, 'Indiana': 3_259_000, 'Iowa': 1_732_000,
    'Kansas': 1_587_000, 'Kentucky': 2_321_000, 'Louisiana': 2_215_000, 'Maine': 856_000, 'Maryland': 3_301_000,
    'Massachusetts': 3_618_000, 'Michigan': 5_797_000, 'Minnesota': 3_255_000, 'Mississippi': 1_572_000, 'Missouri': 3_532_000,
    'Montana': 613_000, 'Nebraska': 933_000, 'Nevada': 1_436_000, 'New Hampshire': 804_000, 'New Jersey': 4_402_000,
    'New Mexico': 1_026_000, 'New York': 8_897_000, 'North Carolina': 4_583_000, 'North Dakota': 418_000, 'Ohio': 5_890_000,
    'Oklahoma': 1_936_000, 'Oregon': 2_581_000, 'Pennsylvania': 7_009_000, 'Rhode Island': 626_000, 'South Carolina': 2_491_000,
    'South Dakota': 460_000, 'Tennessee': 3_467_000, 'Texas': 12_416_000, 'Utah': 1_536_000, 'Vermont': 393_000,
    'Virginia': 4_487_000, 'Washington': 4_140_000, 'West Virginia': 877_000, 'Wisconsin': 3_225_000, 'Wyoming': 274_000,
    'District of Columbia': 500_000
}

In [11]:
candidates:list = ["Total","Donald Trump","Joe Biden"]

In [12]:
election_scores:DataFrame = DataFrame(columns=['Winner','Trump_Electoral_Votes','Biden_Electoral_Votes','Trump_Polling_Error','Biden_Polling_Error','Round_Number'])

In [13]:
for trump_error in range(-4,5,1):
    for biden_error in range(-4,5,1):
        for iteration in range(1,11):
            trump_states_won,biden_states_won,electoral_votes_results,popular_vote_results,state_vote_counts = reset_variables(states,candidates)
            state_polls = set_state_polls(trump_error,biden_error)
            for state in states:
                trump_states_won,biden_states_won,electoral_votes_results,popular_vote_results,state_vote_counts = get_individual_state_votes(state,
                                                        registered_voters,state_polls,states,electoral_votes,
                                                        popular_vote_results,state_vote_counts,electoral_votes_results,
                                                        trump_states_won,biden_states_won,trump_error,biden_error,iteration)
            if(trump_error==-4 and biden_error==-4 and iteration==1):
                get_state_results_df(states,electoral_votes,trump_states_won,state_vote_counts).to_csv(f"States_Trump_Vs_Biden.csv",
                                                    index=True,mode='a',header=True)
            else:
                get_state_results_df(states,electoral_votes,trump_states_won,state_vote_counts).to_csv(f"States_Trump_Vs_Biden.csv",
                                                    index=True,mode='a',header=False)
            national_results = get_national_results_df(electoral_votes_results,popular_vote_results,election_scores,trump_error,biden_error,iteration)
            if(trump_error==-4 and biden_error==-4 and iteration==1):
                DataFrame(national_results[0]).to_csv(f"National_Trump_Vs_Biden.csv",
                                        index=True,
                                        mode='a')
            else:
                DataFrame(national_results[0]).to_csv(f"National_Trump_Vs_Biden.csv",
                                            index=True,
                                            mode='a',header=False)
            election_scores = national_results[1]

Current State: Arizona
States Completed: 2
States Left: 49
Trump Error: -4
Biden Error: 4
Round: 1
